In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import sys
import math
import os

# for measuring the efficiency/variability of the model
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

# importing XGBoost API
from xgboost import XGBRegressor

# obtaining the path to Dataset
current_file = os.path.abspath('')
root_folder = os.path.dirname(current_file)
dataSet_location = os.path.join(root_folder,'Dataset')
data_location = os.path.join(dataSet_location,'processed-data-i1.csv')

# obtaining the dataset
base_data_i1 = pd.read_csv(data_location)

In [109]:
base_data_i1_test = base_data_i1.loc[base_data_i1.Match_id >= 650] 
base_data_i1_train = base_data_i1.loc[base_data_i1.Match_id < 650]

In [68]:
global input_data

def obtain_data_t20_i1(input_data):

    input_over = input_data.iloc[0,0]
    input_wick = input_data.iloc[0,2]
    input_L3Ws = input_data.iloc[0,5]
    input_L3Rs = input_data.iloc[0,6]

    global overs_req
    global wicks_req
    global data_base
    
    # our base dataframe is base_data_i1
    
    # required rows of data to model fromwhich are played within +/- 3 overs of the input_over
    overs_req = (
                 (base_data_i1_train.Over == (input_over-3)) | (base_data_i1_train.Over == (input_over-2)) | 
                 (base_data_i1_train.Over == (input_over-1)) | (base_data_i1_train.Over == (input_over+0)) | 
                 (base_data_i1_train.Over == (input_over+1)) | (base_data_i1_train.Over == (input_over+2)) | 
                 (base_data_i1_train.Over == (input_over+3))
                )
    
    # required rows of data to model from which are played having lost +/- 1 wicket of the input_wick
    wicks_req = ((base_data_i1_train.Inn_Wicks_atm == (input_wick-1)) | (base_data_i1_train.Inn_Wicks_atm == (input_wick)) |
                 (base_data_i1_train.Inn_Wicks_atm == (input_wick+1))
                )
    # required rows of data to model from which the same number of wickets have been lost in the preceeding 3 overs
    L3WLs_req = base_data_i1_train.L3_Wicks == input_L3Ws
    
    # required rows of data to model from which the similar run scoring in the preceeding 3 overs
    L3RSc_req = ((base_data_i1_train.L3_Runs == (input_L3Rs-3)) | (base_data_i1_train.L3_Runs == (input_L3Rs-2)) |
                 (base_data_i1_train.L3_Runs == (input_L3Rs-1)) | (base_data_i1_train.L3_Runs == (input_L3Rs-0)) |
                 (base_data_i1_train.L3_Runs == (input_L3Rs+1)) | (base_data_i1_train.L3_Runs == (input_L3Rs+2)) |
                 (base_data_i1_train.L3_Runs == (input_L3Rs+3))
                )
    
    # extracting required data from the either of the two criteria above
    data_base = base_data_i1_train.loc[ overs_req | wicks_req | L3WLs_req | L3RSc_req ] 
    
    print("Number of instances used for prediction: ",len(data_base))
    
    #calling out the next function
    expected_runs = get_expected_runs(data_base, input_data)
    
    return expected_runs

In [69]:
def get_expected_runs(data_base, input_data):
    # independent and input variables features
    x_features = ['Over','Inn_Score_atm','Inn_Wicks_atm','R_Rate_atm','Ptnr_Avg_atm','L3_Wicks','L3_Runs']
    X = data_base[x_features]
    
    # target variable 
    y = data_base.Runs_to_be_Scored
    
    # calling out the next function
    XGB_model = data_modelling(X,y)
    
    # saving the created model into a package file
#     joblib.dump(XGB_model, '1st_innings_model.pkl') 
    
    #calling out the next function
    expected_runs = obtain_values(XGB_model,X,y, input_data)
    
    return expected_runs

In [70]:
def data_modelling(X,y):
    # programming the model
    XGB_model = XGBRegressor(n_estimators = 400, learning_rate=0.05)
    XGB_model.fit(X,y)
    return XGB_model


In [71]:
def obtain_values(XGB_model,X,y, input_data):
    input_runs = input_data.iloc[0,1]

    # obatining the predictions
    pred_y = XGB_model.predict(input_data)
    
    return pred_y

In [72]:
def RoundedReLU(num):
    if num < 0:
        return 0
    else:
        return round(num)

In [76]:
def print_error_metrics_pred_vs_actual(pred_y, actual_y):
    print("MAE  : ", mean_absolute_error(pred_y, actual_y))
    print("RMSE : ", (mean_squared_error(pred_y, actual_y))**0.5)
    print("MAPE : ", mean_absolute_percentage_error(pred_y, actual_y)*10 ,"%")

### Evaluating Training Error using a Sample of Training Values

In [81]:
train_pred = []
train_actual = []
train_sample_df = base_data_i1_train.sample(frac=0.33, replace=False, random_state=42)
i = 0
for index, row in train_sample_df.iterrows():

    input_data = pd.DataFrame( {'Over' : [row.iloc[4]] , 'Inn_Score_atm' : [row.iloc[5]] , 'Inn_Wicks_atm': [row.iloc[6]],
                            'R_Rate_atm': [row.iloc[7]] ,  'Ptnr_Avg_atm': [row.iloc[8]] , 'L3_Wicks': [row.iloc[-2]],
                            'L3_Runs': [row.iloc[-3]] })
    expected_runs = obtain_data_t20_i1(input_data)[0]
    expected_runs = RoundedReLU(expected_runs)
    train_pred.append(expected_runs)
    train_actual.append(row.iloc[-1])
    i += 1
    print("Predicted ",i," out of ",len(train_sample_df)," samples")

Number of instances used for prediction:  8338
Predicted  1  out of  3593  samples
Number of instances used for prediction:  8106
Predicted  2  out of  3593  samples
Number of instances used for prediction:  7774
Predicted  3  out of  3593  samples
Number of instances used for prediction:  6327
Predicted  4  out of  3593  samples
Number of instances used for prediction:  9311
Predicted  5  out of  3593  samples
Number of instances used for prediction:  9414
Predicted  6  out of  3593  samples
Number of instances used for prediction:  9586
Predicted  7  out of  3593  samples
Number of instances used for prediction:  9153
Predicted  8  out of  3593  samples
Number of instances used for prediction:  7254
Predicted  9  out of  3593  samples
Number of instances used for prediction:  8600
Predicted  10  out of  3593  samples
Number of instances used for prediction:  8757
Predicted  11  out of  3593  samples
Number of instances used for prediction:  9380
Predicted  12  out of  3593  samples
N

Predicted  99  out of  3593  samples
Number of instances used for prediction:  8650
Predicted  100  out of  3593  samples
Number of instances used for prediction:  8719
Predicted  101  out of  3593  samples
Number of instances used for prediction:  9335
Predicted  102  out of  3593  samples
Number of instances used for prediction:  8290
Predicted  103  out of  3593  samples
Number of instances used for prediction:  8692
Predicted  104  out of  3593  samples
Number of instances used for prediction:  10002
Predicted  105  out of  3593  samples
Number of instances used for prediction:  8762
Predicted  106  out of  3593  samples
Number of instances used for prediction:  9221
Predicted  107  out of  3593  samples
Number of instances used for prediction:  4244
Predicted  108  out of  3593  samples
Number of instances used for prediction:  9239
Predicted  109  out of  3593  samples
Number of instances used for prediction:  9688
Predicted  110  out of  3593  samples
Number of instances used fo

Predicted  196  out of  3593  samples
Number of instances used for prediction:  8133
Predicted  197  out of  3593  samples
Number of instances used for prediction:  8596
Predicted  198  out of  3593  samples
Number of instances used for prediction:  6661
Predicted  199  out of  3593  samples
Number of instances used for prediction:  8287
Predicted  200  out of  3593  samples
Number of instances used for prediction:  8215
Predicted  201  out of  3593  samples
Number of instances used for prediction:  8952
Predicted  202  out of  3593  samples
Number of instances used for prediction:  9285
Predicted  203  out of  3593  samples
Number of instances used for prediction:  8699
Predicted  204  out of  3593  samples
Number of instances used for prediction:  9128
Predicted  205  out of  3593  samples
Number of instances used for prediction:  9295
Predicted  206  out of  3593  samples
Number of instances used for prediction:  8708
Predicted  207  out of  3593  samples
Number of instances used fo

Predicted  293  out of  3593  samples
Number of instances used for prediction:  9191
Predicted  294  out of  3593  samples
Number of instances used for prediction:  8614
Predicted  295  out of  3593  samples
Number of instances used for prediction:  7003
Predicted  296  out of  3593  samples
Number of instances used for prediction:  9247
Predicted  297  out of  3593  samples
Number of instances used for prediction:  9409
Predicted  298  out of  3593  samples
Number of instances used for prediction:  9124
Predicted  299  out of  3593  samples
Number of instances used for prediction:  8721
Predicted  300  out of  3593  samples
Number of instances used for prediction:  4181
Predicted  301  out of  3593  samples
Number of instances used for prediction:  8861
Predicted  302  out of  3593  samples
Number of instances used for prediction:  9484
Predicted  303  out of  3593  samples
Number of instances used for prediction:  6989
Predicted  304  out of  3593  samples
Number of instances used fo

Predicted  390  out of  3593  samples
Number of instances used for prediction:  9111
Predicted  391  out of  3593  samples
Number of instances used for prediction:  8447
Predicted  392  out of  3593  samples
Number of instances used for prediction:  9186
Predicted  393  out of  3593  samples
Number of instances used for prediction:  8469
Predicted  394  out of  3593  samples
Number of instances used for prediction:  9010
Predicted  395  out of  3593  samples
Number of instances used for prediction:  9416
Predicted  396  out of  3593  samples
Number of instances used for prediction:  7105
Predicted  397  out of  3593  samples
Number of instances used for prediction:  8575
Predicted  398  out of  3593  samples
Number of instances used for prediction:  8337
Predicted  399  out of  3593  samples
Number of instances used for prediction:  7137
Predicted  400  out of  3593  samples
Number of instances used for prediction:  9468
Predicted  401  out of  3593  samples
Number of instances used fo

...

Predicted  3180  out of  3593  samples
Number of instances used for prediction:  8248
Predicted  3181  out of  3593  samples
Number of instances used for prediction:  9047
Predicted  3182  out of  3593  samples
Number of instances used for prediction:  7995
Predicted  3183  out of  3593  samples
Number of instances used for prediction:  9425
Predicted  3184  out of  3593  samples
Number of instances used for prediction:  10054
Predicted  3185  out of  3593  samples
Number of instances used for prediction:  8708
Predicted  3186  out of  3593  samples
Number of instances used for prediction:  7690
Predicted  3187  out of  3593  samples
Number of instances used for prediction:  6711
Predicted  3188  out of  3593  samples
Number of instances used for prediction:  8344
Predicted  3189  out of  3593  samples
Number of instances used for prediction:  8215
Predicted  3190  out of  3593  samples
Number of instances used for prediction:  8892
Predicted  3191  out of  3593  samples
Number of inst

Predicted  3276  out of  3593  samples
Number of instances used for prediction:  6434
Predicted  3277  out of  3593  samples
Number of instances used for prediction:  9288
Predicted  3278  out of  3593  samples
Number of instances used for prediction:  8377
Predicted  3279  out of  3593  samples
Number of instances used for prediction:  8133
Predicted  3280  out of  3593  samples
Number of instances used for prediction:  8614
Predicted  3281  out of  3593  samples
Number of instances used for prediction:  8997
Predicted  3282  out of  3593  samples
Number of instances used for prediction:  7786
Predicted  3283  out of  3593  samples
Number of instances used for prediction:  9285
Predicted  3284  out of  3593  samples
Number of instances used for prediction:  10038
Predicted  3285  out of  3593  samples
Number of instances used for prediction:  7907
Predicted  3286  out of  3593  samples
Number of instances used for prediction:  9186
Predicted  3287  out of  3593  samples
Number of inst

Predicted  3372  out of  3593  samples
Number of instances used for prediction:  8969
Predicted  3373  out of  3593  samples
Number of instances used for prediction:  9211
Predicted  3374  out of  3593  samples
Number of instances used for prediction:  5473
Predicted  3375  out of  3593  samples
Number of instances used for prediction:  7928
Predicted  3376  out of  3593  samples
Number of instances used for prediction:  9180
Predicted  3377  out of  3593  samples
Number of instances used for prediction:  8708
Predicted  3378  out of  3593  samples
Number of instances used for prediction:  9371
Predicted  3379  out of  3593  samples
Number of instances used for prediction:  8898
Predicted  3380  out of  3593  samples
Number of instances used for prediction:  8337
Predicted  3381  out of  3593  samples
Number of instances used for prediction:  8590
Predicted  3382  out of  3593  samples
Number of instances used for prediction:  8995
Predicted  3383  out of  3593  samples
Number of insta

Predicted  3468  out of  3593  samples
Number of instances used for prediction:  7244
Predicted  3469  out of  3593  samples
Number of instances used for prediction:  9311
Predicted  3470  out of  3593  samples
Number of instances used for prediction:  9019
Predicted  3471  out of  3593  samples
Number of instances used for prediction:  9509
Predicted  3472  out of  3593  samples
Number of instances used for prediction:  7222
Predicted  3473  out of  3593  samples
Number of instances used for prediction:  8094
Predicted  3474  out of  3593  samples
Number of instances used for prediction:  9171
Predicted  3475  out of  3593  samples
Number of instances used for prediction:  9062
Predicted  3476  out of  3593  samples
Number of instances used for prediction:  9187
Predicted  3477  out of  3593  samples
Number of instances used for prediction:  6456
Predicted  3478  out of  3593  samples
Number of instances used for prediction:  8310
Predicted  3479  out of  3593  samples
Number of insta

Predicted  3564  out of  3593  samples
Number of instances used for prediction:  9045
Predicted  3565  out of  3593  samples
Number of instances used for prediction:  8980
Predicted  3566  out of  3593  samples
Number of instances used for prediction:  9205
Predicted  3567  out of  3593  samples
Number of instances used for prediction:  8565
Predicted  3568  out of  3593  samples
Number of instances used for prediction:  8902
Predicted  3569  out of  3593  samples
Number of instances used for prediction:  8819
Predicted  3570  out of  3593  samples
Number of instances used for prediction:  8552
Predicted  3571  out of  3593  samples
Number of instances used for prediction:  8296
Predicted  3572  out of  3593  samples
Number of instances used for prediction:  9172
Predicted  3573  out of  3593  samples
Number of instances used for prediction:  8728
Predicted  3574  out of  3593  samples
Number of instances used for prediction:  7827
Predicted  3575  out of  3593  samples
Number of insta

In [82]:
print_error_metrics_pred_vs_actual(train_pred, train_actual)

MAE  :  10.510993598664069
RMSE :  14.651494653267585
MAPE :  25068742707324.383 %


### Evaluating Testing Error using a Sample of Testing Values

In [75]:
test_pred = []
test_actual = []
test_sample_df = base_data_i1_test.sample(frac=0.5, replace=False, random_state=42)
i = 0
for index, row in test_sample_df.iterrows():

    input_data = pd.DataFrame( {'Over' : [row.iloc[4]] , 'Inn_Score_atm' : [row.iloc[5]] , 'Inn_Wicks_atm': [row.iloc[6]],
                            'R_Rate_atm': [row.iloc[7]] ,  'Ptnr_Avg_atm': [row.iloc[8]] , 'L3_Wicks': [row.iloc[-2]],
                            'L3_Runs': [row.iloc[-3]] })
    expected_runs = obtain_data_t20_i1(input_data)[0]
    expected_runs = RoundedReLU(expected_runs)
    test_pred.append(expected_runs)
    test_actual.append(row.iloc[-1])
    i += 1
    print("Predicted ",i," out of ",len(test_sample_df)," samples")

Number of instances used for prediction:  8773
Predicted  1  out of  1602  samples
Number of instances used for prediction:  7693
Predicted  2  out of  1602  samples
Number of instances used for prediction:  9233
Predicted  3  out of  1602  samples
Number of instances used for prediction:  8542
Predicted  4  out of  1602  samples
Number of instances used for prediction:  5058
Predicted  5  out of  1602  samples
Number of instances used for prediction:  9279
Predicted  6  out of  1602  samples
Number of instances used for prediction:  8821
Predicted  7  out of  1602  samples
Number of instances used for prediction:  8431
Predicted  8  out of  1602  samples
Number of instances used for prediction:  8655
Predicted  9  out of  1602  samples
Number of instances used for prediction:  6614
Predicted  10  out of  1602  samples
Number of instances used for prediction:  6985
Predicted  11  out of  1602  samples
Number of instances used for prediction:  8063
Predicted  12  out of  1602  samples
N

Predicted  99  out of  1602  samples
Number of instances used for prediction:  7457
Predicted  100  out of  1602  samples
Number of instances used for prediction:  8367
Predicted  101  out of  1602  samples
Number of instances used for prediction:  4813
Predicted  102  out of  1602  samples
Number of instances used for prediction:  6934
Predicted  103  out of  1602  samples
Number of instances used for prediction:  9291
Predicted  104  out of  1602  samples
Number of instances used for prediction:  6001
Predicted  105  out of  1602  samples
Number of instances used for prediction:  9290
Predicted  106  out of  1602  samples
Number of instances used for prediction:  8472
Predicted  107  out of  1602  samples
Number of instances used for prediction:  8280
Predicted  108  out of  1602  samples
Number of instances used for prediction:  8949
Predicted  109  out of  1602  samples
Number of instances used for prediction:  9019
Predicted  110  out of  1602  samples
Number of instances used for

Predicted  196  out of  1602  samples
Number of instances used for prediction:  9301
Predicted  197  out of  1602  samples
Number of instances used for prediction:  8601
Predicted  198  out of  1602  samples
Number of instances used for prediction:  8757
Predicted  199  out of  1602  samples
Number of instances used for prediction:  8872
Predicted  200  out of  1602  samples
Number of instances used for prediction:  3961
Predicted  201  out of  1602  samples
Number of instances used for prediction:  8815
Predicted  202  out of  1602  samples
Number of instances used for prediction:  7504
Predicted  203  out of  1602  samples
Number of instances used for prediction:  8572
Predicted  204  out of  1602  samples
Number of instances used for prediction:  9147
Predicted  205  out of  1602  samples
Number of instances used for prediction:  8868
Predicted  206  out of  1602  samples
Number of instances used for prediction:  8099
Predicted  207  out of  1602  samples
Number of instances used fo

Predicted  293  out of  1602  samples
Number of instances used for prediction:  5176
Predicted  294  out of  1602  samples
Number of instances used for prediction:  9315
Predicted  295  out of  1602  samples
Number of instances used for prediction:  5718
Predicted  296  out of  1602  samples
Number of instances used for prediction:  8978
Predicted  297  out of  1602  samples
Number of instances used for prediction:  8993
Predicted  298  out of  1602  samples
Number of instances used for prediction:  8089
Predicted  299  out of  1602  samples
Number of instances used for prediction:  8898
Predicted  300  out of  1602  samples
Number of instances used for prediction:  7706
Predicted  301  out of  1602  samples
Number of instances used for prediction:  8448
Predicted  302  out of  1602  samples
Number of instances used for prediction:  9094
Predicted  303  out of  1602  samples
Number of instances used for prediction:  6273
Predicted  304  out of  1602  samples
Number of instances used fo

Predicted  390  out of  1602  samples
Number of instances used for prediction:  6864
Predicted  391  out of  1602  samples
Number of instances used for prediction:  8294
Predicted  392  out of  1602  samples
Number of instances used for prediction:  8876
Predicted  393  out of  1602  samples
Number of instances used for prediction:  8819
Predicted  394  out of  1602  samples
Number of instances used for prediction:  3580
Predicted  395  out of  1602  samples
Number of instances used for prediction:  8391
Predicted  396  out of  1602  samples
Number of instances used for prediction:  9013
Predicted  397  out of  1602  samples
Number of instances used for prediction:  9339
Predicted  398  out of  1602  samples
Number of instances used for prediction:  9155
Predicted  399  out of  1602  samples
Number of instances used for prediction:  9279
Predicted  400  out of  1602  samples
Number of instances used for prediction:  8540
Predicted  401  out of  1602  samples
Number of instances used fo

...

Predicted  1164  out of  1602  samples
Number of instances used for prediction:  9605
Predicted  1165  out of  1602  samples
Number of instances used for prediction:  8053
Predicted  1166  out of  1602  samples
Number of instances used for prediction:  9170
Predicted  1167  out of  1602  samples
Number of instances used for prediction:  8722
Predicted  1168  out of  1602  samples
Number of instances used for prediction:  9139
Predicted  1169  out of  1602  samples
Number of instances used for prediction:  8435
Predicted  1170  out of  1602  samples
Number of instances used for prediction:  8133
Predicted  1171  out of  1602  samples
Number of instances used for prediction:  8500
Predicted  1172  out of  1602  samples
Number of instances used for prediction:  9522
Predicted  1173  out of  1602  samples
Number of instances used for prediction:  8310
Predicted  1174  out of  1602  samples
Number of instances used for prediction:  8216
Predicted  1175  out of  1602  samples
Number of insta

Predicted  1260  out of  1602  samples
Number of instances used for prediction:  6599
Predicted  1261  out of  1602  samples
Number of instances used for prediction:  8522
Predicted  1262  out of  1602  samples
Number of instances used for prediction:  7807
Predicted  1263  out of  1602  samples
Number of instances used for prediction:  5628
Predicted  1264  out of  1602  samples
Number of instances used for prediction:  9547
Predicted  1265  out of  1602  samples
Number of instances used for prediction:  8731
Predicted  1266  out of  1602  samples
Number of instances used for prediction:  9273
Predicted  1267  out of  1602  samples
Number of instances used for prediction:  8847
Predicted  1268  out of  1602  samples
Number of instances used for prediction:  9241
Predicted  1269  out of  1602  samples
Number of instances used for prediction:  8759
Predicted  1270  out of  1602  samples
Number of instances used for prediction:  7461
Predicted  1271  out of  1602  samples
Number of insta

Predicted  1356  out of  1602  samples
Number of instances used for prediction:  7566
Predicted  1357  out of  1602  samples
Number of instances used for prediction:  9256
Predicted  1358  out of  1602  samples
Number of instances used for prediction:  8299
Predicted  1359  out of  1602  samples
Number of instances used for prediction:  8753
Predicted  1360  out of  1602  samples
Number of instances used for prediction:  8060
Predicted  1361  out of  1602  samples
Number of instances used for prediction:  5865
Predicted  1362  out of  1602  samples
Number of instances used for prediction:  9170
Predicted  1363  out of  1602  samples
Number of instances used for prediction:  8514
Predicted  1364  out of  1602  samples
Number of instances used for prediction:  9368
Predicted  1365  out of  1602  samples
Number of instances used for prediction:  8692
Predicted  1366  out of  1602  samples
Number of instances used for prediction:  9211
Predicted  1367  out of  1602  samples
Number of insta

Predicted  1452  out of  1602  samples
Number of instances used for prediction:  8566
Predicted  1453  out of  1602  samples
Number of instances used for prediction:  9091
Predicted  1454  out of  1602  samples
Number of instances used for prediction:  8662
Predicted  1455  out of  1602  samples
Number of instances used for prediction:  8561
Predicted  1456  out of  1602  samples
Number of instances used for prediction:  9285
Predicted  1457  out of  1602  samples
Number of instances used for prediction:  9188
Predicted  1458  out of  1602  samples
Number of instances used for prediction:  7823
Predicted  1459  out of  1602  samples
Number of instances used for prediction:  6240
Predicted  1460  out of  1602  samples
Number of instances used for prediction:  8966
Predicted  1461  out of  1602  samples
Number of instances used for prediction:  7316
Predicted  1462  out of  1602  samples
Number of instances used for prediction:  9241
Predicted  1463  out of  1602  samples
Number of insta

Predicted  1548  out of  1602  samples
Number of instances used for prediction:  8772
Predicted  1549  out of  1602  samples
Number of instances used for prediction:  9020
Predicted  1550  out of  1602  samples
Number of instances used for prediction:  8704
Predicted  1551  out of  1602  samples
Number of instances used for prediction:  5965
Predicted  1552  out of  1602  samples
Number of instances used for prediction:  7665
Predicted  1553  out of  1602  samples
Number of instances used for prediction:  9145
Predicted  1554  out of  1602  samples
Number of instances used for prediction:  8567
Predicted  1555  out of  1602  samples
Number of instances used for prediction:  9014
Predicted  1556  out of  1602  samples
Number of instances used for prediction:  9050
Predicted  1557  out of  1602  samples
Number of instances used for prediction:  8484
Predicted  1558  out of  1602  samples
Number of instances used for prediction:  7995
Predicted  1559  out of  1602  samples
Number of insta

In [79]:
print_error_metrics_pred_vs_actual(test_pred, test_actual)

MAE  :  12.863920099875156
RMSE :  18.1869882559681
MAPE :  2.1178969553057354 %
